In [ ]:
#reading represenatative patents
import pandas as pd
data=pd.read_csv("patstat_pat_no.csv")
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted
0,574528.0,307102421.0,1985-04-10,EP,Y
1,574586.0,387396319.0,1987-06-08,EP,Y
2,574660.0,387398325.0,1986-08-04,EP,Y
3,1226456.0,303518198.0,1991-08-29,EP,Y
4,1226485.0,302640285.0,1992-08-12,EP,Y


In [13]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    hosts=f"https://es2.logic-mill.net:443",
    basic_auth=("elastic", "MTqCwEli2DWAJTFGKwIa"),
    retry_on_status=[500, 501, 502, 503, 504, 524],
    retry_on_timeout=True,
    request_timeout=60,
)
res = client.get(
    index='patents',
    id=574528,
)

print(res["_source"].keys())

dict_keys(['appln_auth', 'earliest_filing_date', 'cpc_classes', 'appln_id', 'earliest_priority_date', 'abstract', 'title', 'patspecter_embedding', 'paecter_embedding', 'publno'])


In [14]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    hosts=f"https://es2.logic-mill.net:443",
    basic_auth=("elastic", "MTqCwEli2DWAJTFGKwIa"),
    retry_on_status=[500, 501, 502, 503, 504, 524],
    retry_on_timeout=True,
    request_timeout=60,
)
res = client.get(
    index='patents',
    id=574528,
    _source=["paecter_embedding"]
)
embedding = res["_source"]["paecter_embedding"]
k = 10
query = {
        "field": "paecter_embedding",
        "num_candidates": min(k * 2, 10_000),
        "query_vector": embedding
    }
search = client.search(
    index="patents",
    knn=query,
    source=False,
    size=k
)
print(search)
len(search["hits"]["hits"])

{'took': 22, 'timed_out': False, '_shards': {'total': 32, 'successful': 32, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10, 'relation': 'eq'}, 'max_score': 0.999717, 'hits': [{'_index': 'patents_v2', '_id': '574528', '_score': 0.999717, '_ignored': ['abstract.keyword']}, {'_index': 'patents_v2', '_id': '74046750', '_score': 0.986259}, {'_index': 'patents_v2', '_id': '11376922', '_score': 0.98601913, '_ignored': ['abstract.keyword']}, {'_index': 'patents_v2', '_id': '22906565', '_score': 0.9859996}, {'_index': 'patents_v2', '_id': '9919605', '_score': 0.98561096}, {'_index': 'patents_v2', '_id': '7697035', '_score': 0.9855838}, {'_index': 'patents_v2', '_id': '79283550', '_score': 0.9847808}, {'_index': 'patents_v2', '_id': '50404595', '_score': 0.9847162}, {'_index': 'patents_v2', '_id': '48747291', '_score': 0.9843309}, {'_index': 'patents_v2', '_id': '50549125', '_score': 0.9842887, '_ignored': ['abstract.keyword']}]}}


10

In [ ]:
# Fetch the embedding for the given patent ID
res = client.get(
    index='patents',
    id=574528,
)
embedding = res["_source"]["paecter_embedding"]
k = 2000
target_date = res["_source"]["earliest_priority_date"]  #priority date of the focal patent

# Construct the search query with KNN and date filter
query = {
    "size": k,
    "query": {
        "bool": {
            "must": [
                {  # kNN Search
                    "knn": {
                        "field": "paecter_embedding",
                        "num_candidates": min(k * 2, 10_000),
                        "query_vector": embedding
                    }
                },
                {  # Date Filter
                    "range": {
                        "earliest_filing_date": {
                            "lte": target_date
                        }
                    }
                }
            ]
        }
    },
    "_source": ["earliest_filing_date"]
}


# Perform the search
search = client.search(
    index="patents",
    body=query
)
# Print the search results
result=search["hits"]["hits"]
print(result)
print(len(result))

[{'_index': 'patents_v2', '_id': '574528', '_score': 1.999717, '_ignored': ['abstract.keyword'], '_source': {'earliest_filing_date': '1985-04-10'}}, {'_index': 'patents_v2', '_id': '9306402', '_score': 1.9822814, '_source': {'earliest_filing_date': '1984-07-20'}}, {'_index': 'patents_v2', '_id': '8406553', '_score': 1.9822466, '_ignored': ['abstract.keyword'], '_source': {'earliest_filing_date': '1980-01-11'}}, {'_index': 'patents_v2', '_id': '8194189', '_score': 1.9820185, '_source': {'earliest_filing_date': '1985-04-04'}}, {'_index': 'patents_v2', '_id': '8414205', '_score': 1.98196, '_source': {'earliest_filing_date': '1981-04-04'}}, {'_index': 'patents_v2', '_id': '8374596', '_score': 1.9817214, '_ignored': ['abstract.keyword'], '_source': {'earliest_filing_date': '1975-01-16'}}, {'_index': 'patents_v2', '_id': '20348492', '_score': 1.9811366, '_ignored': ['abstract.keyword'], '_source': {'earliest_filing_date': '1982-11-08'}}, {'_index': 'patents_v2', '_id': '8406799', '_score': 1

In [17]:
import numpy as np
data['similar_patents']=np.nan
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents
0,574528.0,307102421.0,1985-04-10,EP,Y,NaN
1,574586.0,387396319.0,1987-06-08,EP,Y,NaN
2,574660.0,387398325.0,1986-08-04,EP,Y,NaN
3,1226456.0,303518198.0,1991-08-29,EP,Y,NaN
4,1226485.0,302640285.0,1992-08-12,EP,Y,NaN


In [18]:
data['similar_patents']=data['similar_patents'].astype('object')
ids=list(data['docdb_family_id'])
ids

[574528.0,
 574586.0,
 574660.0,
 1226456.0,
 1226485.0,
 1236211.0,
 1236234.0,
 1236542.0,
 1239366.0,
 1239622.0,
 1239657.0,
 1239741.0,
 1239749.0,
 1239760.0,
 1340219.0,
 1340485.0,
 1340488.0,
 1340519.0,
 1340681.0,
 1341021.0,
 1762664.0,
 1763206.0,
 3411930.0,
 3460581.0,
 3460621.0,
 3460833.0,
 3460835.0,
 3460912.0,
 3460975.0,
 3461011.0,
 3461021.0,
 3461121.0,
 3461123.0,
 3478350.0,
 3478351.0,
 3478506.0,
 3478541.0,
 3478788.0,
 3478811.0,
 3478851.0,
 3478868.0,
 3478887.0,
 3479022.0,
 3479063.0,
 3479143.0,
 3479187.0,
 3479252.0,
 3479254.0,
 3479275.0,
 3479294.0,
 3479303.0,
 3479359.0,
 3479376.0,
 3479408.0,
 3479415.0,
 3479450.0,
 3479467.0,
 3479475.0,
 3479498.0,
 3479517.0,
 3479590.0,
 3479671.0,
 3479746.0,
 3479774.0,
 3479780.0,
 3479803.0,
 3479805.0,
 3479828.0,
 3479832.0,
 3479837.0,
 3479858.0,
 3479871.0,
 3479909.0,
 3479918.0,
 3479935.0,
 3479986.0,
 3480016.0,
 3480018.0,
 3480053.0,
 3480092.0,
 3480157.0,
 3480195.0,
 3480208.0,
 348023

In [ ]:
error_ids=[]
h=-1
for i in ids:
    try:
        h=h+1
        # Fetch the embedding for the given patent ID
        res = client.get(
            index='patents',
            id=int(i),
        )
        embedding = res["_source"]["paecter_embedding"]
        k = 2000
        target_date = res["_source"]["earliest_priority_date"]  #priority date of the focal patent

        # Construct the search query with KNN and date filter
        query = {
            "size": k,
            "query": {
                "bool": {
                    "must": [
                        {  # kNN Search
                            "knn": {
                                "field": "paecter_embedding",
                                "num_candidates": min(k * 2, 10_000),  # Increase num_candidates
                                "query_vector": embedding
                            }
                        },
                        {  # Date Filter
                            "range": {
                                "earliest_filing_date": {
                                    "lte": target_date
                                }
                            }
                        }
                    ]
                }
            },
            "_source": ["earliest_filing_date"]
        }


        # Perform the search
        search = client.search(
            index="patents",
            body=query
        )
        # Print the search results
        result=search["hits"]["hits"]
    
        l=[]
        for j in search["hits"]["hits"]:
            l.append((j['_id'],j["_score"]))
        data.at[h,'similar_patents']=l
    except:
        error_ids.append(i)

In [20]:
data.head()

,docdb_family_id,earliest_pat_publn_id,earliest_filing_date,appln_auth,granted,similar_patents
0,574528.0,307102421.0,1985-04-10,EP,Y,"[(574528, 1.999717), (9306402, 1.9822814), (84..."
1,574586.0,387396319.0,1987-06-08,EP,Y,"[(574586, 1.9997215), (63363761, 1.97839), (83..."
2,574660.0,387398325.0,1986-08-04,EP,Y,"[(574660, 1.9997239), (6240233, 1.9768794), (2..."
3,1226456.0,303518198.0,1991-08-29,EP,Y,"[(1226456, 1.9997258), (4856238, 1.9822338), (..."
4,1226485.0,302640285.0,1992-08-12,EP,Y,"[(1226485, 1.9996202), (6328120, 1.987576), (2..."


In [21]:
error_ids

[1340219.0,
 1762664.0,
 3684009.0,
 4151350.0,
 4154642.0,
 4173026.0,
 4234192.0,
 5331371.0,
 6460999.0,
 6488658.0,
 8226697.0,
 10941060.0,
 10974889.0,
 10975865.0,
 10975934.0,
 10975949.0,
 10976022.0,
 10976106.0,
 10976237.0,
 10976240.0,
 10976259.0,
 10976507.0,
 10976637.0,
 10976895.0,
 10976918.0,
 10987832.0,
 10988864.0,
 10989750.0,
 10989891.0,
 10991123.0,
 10991361.0,
 10991559.0,
 10991751.0,
 10991906.0,
 10991983.0,
 10992158.0,
 10992737.0,
 25643279.0,
 25676654.0,
 25677152.0,
 25679787.0,
 25685277.0,
 26317962.0,
 26317988.0,
 26318248.0,
 26318329.0,
 27153130.0,
 27171141.0,
 27204425.0,
 27427485.0,
 27570108.0,
 27582829.0,
 27736263.0,
 28052963.0,
 29588444.0,
 30002014.0,
 31898406.0,
 31898420.0,
 31898428.0,
 32012037.0,
 32012048.0,
 32398610.0,
 32684040.0,
 32731273.0,
 32777522.0,
 33453864.0,
 33494530.0,
 33542404.0,
 34113335.0,
 34674455.0,
 34951488.0,
 35185390.0,
 35735318.0,
 35902039.0,
 36659913.0,
 37106903.0,
 37492567.0,
 37499822.

In [ ]:
data.to_pickle("21similar_patents_updated.pkl")

In [ ]:
#storing the patents for which logic mill server gave error fetching
import pickle
with open("21error_ids.pkl", "wb") as f:
    pickle.dump(error_ids, f)

In [ ]:
pd.read_pickle("21error_ids.pkl")

[1340219.0,
 1762664.0,
 3684009.0,
 4151350.0,
 4154642.0,
 4173026.0,
 4234192.0,
 5331371.0,
 6460999.0,
 6488658.0,
 8226697.0,
 10941060.0,
 10974889.0,
 10975865.0,
 10975934.0,
 10975949.0,
 10976022.0,
 10976106.0,
 10976237.0,
 10976240.0,
 10976259.0,
 10976507.0,
 10976637.0,
 10976895.0,
 10976918.0,
 10987832.0,
 10988864.0,
 10989750.0,
 10989891.0,
 10991123.0,
 10991361.0,
 10991559.0,
 10991751.0,
 10991906.0,
 10991983.0,
 10992158.0,
 10992737.0,
 25643279.0,
 25676654.0,
 25677152.0,
 25679787.0,
 25685277.0,
 26317962.0,
 26317988.0,
 26318248.0,
 26318329.0,
 27153130.0,
 27171141.0,
 27204425.0,
 27427485.0,
 27570108.0,
 27582829.0,
 27736263.0,
 28052963.0,
 29588444.0,
 30002014.0,
 31898406.0,
 31898420.0,
 31898428.0,
 32012037.0,
 32012048.0,
 32398610.0,
 32684040.0,
 32731273.0,
 32777522.0,
 33453864.0,
 33494530.0,
 33542404.0,
 34113335.0,
 34674455.0,
 34951488.0,
 35185390.0,
 35735318.0,
 35902039.0,
 36659913.0,
 37106903.0,
 37492567.0,
 37499822.